## AGIにおける計算の各部分のベンチマークを計測する。

In [8]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation

from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy import Function
from sympy import Matrix

edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64) 
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",") 
n = len(P)
L =np.genfromtxt('csv/eigVals.csv', delimiter=",") 
L_pos = np.array([L[i] if L[i]>0 else 0 for i in range(n)])
d = np.count_nonzero(L_pos)	# d ... the number of positive values
Ln = np.sqrt(L_pos)

f2 = np.array(Ln[0:d])
f2[::2] = 0
f1 = Ln[0:d] - f2
e1 = f1 / np.linalg.norm(f1)
e2 = f2 / np.linalg.norm(f2)
temp1 = e1
temp2 = e2

Xs = np.array([])
Ys = np.array([])

for i in np.arange(n):
    p0 = P[i,0:d]
    Xs = np.append(Xs,np.dot(p0,e1))
    Ys = np.append(Ys,np.dot(p0,e2))
    
a1,b1,c1,a2,b2,c2,t,s = sp.symbols('a1 b1 c1 a2 b2 c2 t s')   # variables
x2_s,y2_s = sp.symbols('x2_s y2_s') # values
P_i = sp.MatrixSymbol('P_i', d, 1)
E1 = sp.MatrixSymbol('E1', d, 1)
E2 = sp.MatrixSymbol('E2', d, 1)
var = (x2_s,y2_s,P_i,E1,E2,a1,b1,c1,a2,b2,c2,t,s)

_E1 = a1*sp.Matrix(E1) + b1*sp.Matrix(E2) + c1*sp.Matrix(P_i)
_E2 = a2*sp.Matrix(E1) + b2*sp.Matrix(E2) + c2*sp.Matrix(P_i)
R  = s*sp.Matrix(E1) + t*sp.Matrix(E2)

f = Matrix([
		_E1.dot(_E1) - 1,
		_E2.dot(_E2) - 1,
		_E1.dot(_E2),
		R.dot(R) - 1,
		_E1.dot(R) - sp.Matrix(E1).dot(R),
		_E2.dot(R) - sp.Matrix(E2).dot(R),
		sp.Matrix(P_i).dot(_E1) - x2_s,
		sp.Matrix(P_i).dot(_E2) - y2_s
		])

lam_f = lambdify(var, f, 'numpy')

def lam_f2(x2,y2,p,e1,e2):
    return lambda a1,b1,c1,a2,b2,c2,t,s: np.linalg.norm(lam_f(x2,y2,sp.Matrix(p),sp.Matrix(e1),sp.Matrix(e2),a1,b1,c1,a2,b2,c2,t,s))

f2 = lam_f2(1,1,P[0],e1,e2)
def g(args): return f2(*args)
res = opt.minimize(g,[1,0,0,0,1,0,1,1],method='L-BFGS-B')
e1 = res.x[0]*temp1 + res.x[1]*temp2 + res.x[2]*P[0]
e2 = res.x[3]*temp1 + res.x[4]*temp2 + res.x[5]*P[0]
for i in np.arange(n):
    p0 = P[i]
    global Xs, Ys
    Xs[i] = np.dot(p0,e1)
    Ys[i] = np.dot(p0,e2)

In [7]:
X_sample = 3 * np.random.random_sample((10, 1)) - 1.5
Y_sample = 3 * np.random.random_sample((10, 1)) - 1.5
varLambdify = (a1, b1, c1, a2, b2, c2, s, t)
func = sp.Matrix.norm(f)
# V: vector symbol, Vv:np.ndarray
def subV(e, V, Vv):
    for i in range(len(Vv)):
            e = e.subs(V[i,0], Vv[i])
    return e
# Vs, Vvs : list
def subV_mul(e,Vs,Vvs):
    if len(Vvs) > 0:
        if isinstance(Vvs[0], np.ndarray):
            if(len(Vvs[0])>1):
                return subV_mul(subV(e,Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
            else:
                return subV_mul(e.subs(Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
        else:
            return subV_mul(e.subs(Vs[0],Vvs[0]),Vs[1:],Vvs[1:])
    return e

import time
start = time.time()

for i in range(1):
    lamlam = lambdify(varLambdify,subV_mul(func,[x2_s,y2_s,P_i,E1,E2],[X_sample[i],Y_sample[i],P[0],e1,e2]),'numpy')

elapsed_time = time.time() - start
print("elapsed_time: " + str(elapsed_time) + "[sec]")

In [ ]:
import time

start = time.time()

for i in range(100):

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")

In [ ]:
import time

start = time.time()

for i in range(100):
    res = opt.minimize

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")

In [ ]:
e1s,e2s = np.array([])
for i in range(100):
    res = opt.minimize()

In [ ]:
import time

start = time.time()

for k in range(100):
    for i in range(n):
        Xs[i] = np.dot(p0,e1s[k])
        Ys[i] = np.dot(p0,e2s[k])

elapsed_time = time.time() - start
print ("elapsed_time: " + str(elapsed_time) + "[sec]")